In [7]:
from langchain_core.prompts import ChatPromptTemplate

import os
from dotenv import load_dotenv

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

True

In [8]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [9]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

messages=[HumanMessage(content='Tell me an interesting fact about dog')]


In [10]:
print(prompt_val.to_messages())

[HumanMessage(content='Tell me an interesting fact about dog')]


In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
result = model.invoke(prompt_val)
result

AIMessage(content="Dogs have a sense of time and can actually sense how much time has passed. They are able to pick up on cues such as the length of shadows, changes in light, and the smell of different times of day to determine how much time has elapsed. This is why they may know when it's time for their owners to come home or when it's time for their next meal.", response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 14, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

"Dogs have a sense of time and can actually sense how much time has passed. They are able to pick up on cues such as the length of shadows, changes in light, and the smell of different times of day to determine how much time has elapsed. This is why they may know when it's time for their owners to come home or when it's time for their next meal."

### Now let´s do this LCEL

In [13]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

basicchain = model | output_parser

In [14]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [15]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

'Dogs have an incredible sense of smell, with some breeds having up to 300 million scent receptors in their noses (compared to about 5 million in humans). This allows them to detect certain diseases in humans, such as cancer and diabetes, by simply sniffing their breath or skin.'

### Retrieval Augmented Generation with LCEL

In [16]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [17]:
retriever.get_relevant_documents("What does the dog want to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]

In [18]:
retriever.invoke("What does the dog want to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]

In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [20]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The dog likes to eat pizza.'

In [22]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
retrieval_chain.invoke("What does the dog like to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The dog likes to eat pizza.'

In [24]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(upper=lambda x: x["input"].upper()),
    modified=lambda x: x["input"] * 3,
)

runnable.invoke({"input": "hello!"})

{'passed': {'input': 'hello!'},
 'extra': {'input': 'hello!', 'upper': 'HELLO!'},
 'modified': 'hello!hello!hello!'}